In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from matplotlib import ticker
import time
import warnings
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:f}'.format)
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('train.csv')
df.drop(['Unnamed: 0','id'],axis=1,inplace=True)
df.rename({'satisfaction':'Satisfaction'},axis=1,inplace=True)

In [3]:
df.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,Satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.000000,neutral or dissatisfied
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.000000,neutral or dissatisfied
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.000000,satisfied
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.000000,neutral or dissatisfied
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.000000,satisfied


In [4]:
df = df.fillna(0)

In [1]:
cont_features = ['Flight Distance','Arrival Delay in Minutes']

In [2]:
cat_features = ['Customer Type',
 'Type of Travel',
 'Class',
 'Departure/Arrival time convenient',
 'Ease of Online booking',
'Inflight wifi service',
 'Online boarding',
 'Seat comfort',
 'Inflight entertainment',
 'On-board service',
 'Leg room service',
 'Baggage handling',
 'Checkin service',
 'Inflight service',
 'Cleanliness']

selected_feature=['Flight Distance', 'Arrival Delay in Minutes',
       'Customer Type',
       'Type of Travel', 'Class', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Online boarding', 'Inflight entertainment', 'On-board service',
       'Leg room service', 'Baggage handling', 'Checkin service',
       'Inflight service', 'Cleanliness','Seat comfort']

In [7]:
resp = pd.DataFrame(df.iloc[0]).T
resp = resp.drop(['Satisfaction'],axis=1)

In [8]:
resp = resp[selected_feature]

### Preprocessing

In [3]:
cont_features = ['Flight Distance','Arrival Delay in Minutes']

In [4]:
import pickle
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler

# Load OneHotEncoder
with open("onehotencoder.pkl", "rb") as f:
    onehotencoder = pickle.load(f)

# Load LabelEncoder
with open("labelencoder.pkl", "rb") as f:
    labelencoder = pickle.load(f)

# Load StandardScaler
with open("standardscaler.pkl", "rb") as f:
    scaler = pickle.load(f)

model = CatBoostClassifier()
model.load_model("model.cb")

In [5]:
def preprocess_data(resp):
    # Create new feature
    resp['Seat comfort high rating'] = np.where(resp['Seat comfort']>3,1,0)
    resp['Inflight entertainment high rating'] = np.where(resp['Inflight entertainment']>3,1,0)
    resp['Leg room high rating'] = np.where(resp['Leg room service']>3,1,0)

    resp['Flight Distance'] = resp['Flight Distance']**0.5
    resp['Arrival Delay in Minutes'] = np.log1p(float(resp['Arrival Delay in Minutes'])**0.5)

    # Scale
    resp_scaled = scaler.fit_transform(resp[cont_features])
    resp_scaled = pd.DataFrame(resp_scaled, columns=cont_features).reset_index(drop=True)

    # One Hot Encoding
    resp_ohe = pd.DataFrame()
    resp_ohe['Customer Type_disloyal Customer'] = np.where(resp['Customer Type']=='Loyal Customer',0,1)
    resp_ohe['Type of Travel_Personal Travel'] = np.where(resp['Type of Travel']=='Business Travel',0,1)

    # Label Encoding
    resp_le = labelencoder.fit_transform(resp['Class'])
    resp_le = pd.DataFrame(resp_le, columns=['Class'])

    # Concatenate back
    f=['Class','Customer Type', 'Type of Travel','Flight Distance','Arrival Delay in Minutes']
    res = resp.reset_index(drop=True)
    res.drop(f,axis=1,inplace=True)
    resp_cleaned = pd.concat([resp_scaled,resp_ohe,resp_le,res],axis=1).drop('Seat comfort',axis=1)
    
    return resp_cleaned


In [6]:
resp_prepro = preprocess_data(resp)

NameError: name 'resp' is not defined

In [ ]:
model.predict(resp_prepro)

In [9]:
import gradio as gr

def predict_flight_satisfaction(flight_distance, arrival_delay, customer_type, travel_type, flight_class, wifi_service, 
                                time_convenience, online_booking, boarding, entertainment, onboard_service, leg_room_service, 
                                baggage_handling, checkin_service, inflight_service, cleanliness, seat_comfort):
    
    data = {
        "Flight Distance": [flight_distance],
        "Arrival Delay in Minutes": [arrival_delay],
        "Customer Type": [customer_type],
        "Type of Travel": [travel_type],
        "Class": [flight_class],
        "Inflight wifi service": [float(wifi_service)],
        "Departure/Arrival time convenient": [float(time_convenience)],
        "Ease of Online booking": [float(online_booking)],
        "Online boarding": [float(boarding)],
        "Inflight entertainment": [float(entertainment)],
        "On-board service": [float(onboard_service)],
        "Leg room service": [float(leg_room_service)],
        "Baggage handling": [float(baggage_handling)],
        "Checkin service": [float(checkin_service)],
        "Inflight service": [float(inflight_service)],
        "Cleanliness": [float(cleanliness)],
        "Seat comfort": [float(seat_comfort)],

    }

    df = pd.DataFrame(data)
    resp = preprocess_data(df)
    pred = model.predict(resp)
    
    if(pred==0):
        text = "Neutral or Dissatisfied"
    else:
        text = "Satisfied"

    return text

# Create the input components
flight_distance_input = gr.inputs.Number(label="Flight Distance")
arrival_delay_input = gr.inputs.Number(label="Arrival Delay in Minutes")
customer_type_input = gr.inputs.Dropdown(choices=["Loyal Customer", "Disloyal Customer"], label="Customer Type")
travel_type_input = gr.inputs.Dropdown(choices=["Personal Travel", "Business Travel"], label="Type of Travel")
flight_class_input = gr.inputs.Dropdown(choices=["Eco", "Eco Plus", "Business"], label="Class")
wifi_service_input = gr.inputs.Slider(minimum=1, maximum=5, default=3, label="Inflight wifi service")
time_convenience_input = gr.inputs.Slider(minimum=1, maximum=5, default=3, label="Departure/Arrival time convenient")
online_booking_input = gr.inputs.Slider(minimum=1, maximum=5, default=3, label="Ease of Online booking")
boarding_input = gr.inputs.Slider(minimum=1, maximum=5, default=3, label="Online boarding")
entertainment_input = gr.inputs.Slider(minimum=1, maximum=5, default=3, label="Inflight entertainment")
onboard_service_input = gr.inputs.Slider(minimum=1, maximum=5, default=3, label="On-board service")
leg_room_service_input = gr.inputs.Slider(minimum=1, maximum=5, default=3, label="Leg room service")
baggage_handling_input = gr.inputs.Slider(minimum=1, maximum=5, default=3, label="Baggage handling")
checkin_service_input = gr.inputs.Slider(minimum=1, maximum=5, default=3, label="Checkin service")
inflight_service_input = gr.inputs.Slider(minimum=1, maximum=5, default=3, label="Inflight service")
cleanliness_input = gr.inputs.Slider(minimum=1, maximum=5, default=3, label="Cleanliness")
seat_comfort_input = gr.inputs.Slider(minimum=1, maximum=5, default=3, label="Seat comfort")

# Create the output component
output_text = gr.outputs.Textbox(label="Average Numerical Features")

# Create the interface
gr.Interface(
    fn=predict_flight_satisfaction,
    inputs=[
        flight_distance_input,
        arrival_delay_input,
        customer_type_input,
        travel_type_input,
        flight_class_input,
        wifi_service_input,
        time_convenience_input,
        online_booking_input,
        boarding_input,
        entertainment_input,
        onboard_service_input,
        leg_room_service_input,
        baggage_handling_input,
        checkin_service_input,
        inflight_service_input,
        cleanliness_input,
        seat_comfort_input
    ],
    outputs=output_text,
    title="Flight Satisfaction Predictor",
    description="Predicts flight satisfaction based on various factors"
).launch(share=True)


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://5b5d85992c5a450266.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
